*   실행환경: 구글 코랩
*   작성일: 231001
*   작성자: 이혜원



장르별 스테디셀러 크롤링

https://www.melon.com/genre/song_list.htm?gnrCode=GN0100

장르 8개, 각 500개씩 총 4000개 정보를 가져옴

장르
* 'GN0100': 발라드
* 'GN0200': 댄스
* 'GN0300': 랩/힙합
* 'GN0400': R&B/Soul
* 'GN0500': 인디음악
* 'GN0600': 록/메탈
* 'GN0700': 트로트
* 'GN0800': 포크/블루스


In [ ]:
# rank: 순위
# song_name: 곡 제목
# artist: 가수명
# likes: 좋아요 수
# 앨범ID: album_id
# 가사ID: song_id
def make_info_list(tr_lst, gnrCode):
  all_list = []

  for tr in tr_lst:
    rank = tr.select_one(".rank").text
    song_id = tr.select_one("div.wrap a.song_info")['href'][36:-3]
    album_id = tr.select_one("div.wrap a.image_typeAll")['href'][37:-3]
    song_name = tr.select_one("div.rank01").text.strip()

    artist = tr.select_one("div.rank02 a")
    if artist is not None:
      artist = artist.text.strip()

    # 좋아요 건수가 0건이라고 나옴 -> get 요청으로 해서 그런듯
    # likes = tr.select_one("button.like > span.cnt").text.strip()

    one_list = [gnrCode, rank, song_name, artist, album_id, song_id]
    all_list.append(one_list)

  return all_list

In [ ]:
import pandas as pd
import re
import time
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

In [ ]:
def make_album_info_dict(gnrCodeList):
    all_dict = {}
    headers ={'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
              'referer': 'https://www.melon.com/genre/song_list.htm?gnrCode=GN0100&steadyYn=Y&startIndex=1'}

    lst = []
    for gnrCode in gnrCodeList:
        page_num = 10
        start_index_list = [n*50+1 for n in range(page_num)]
        for start_index in start_index_list:
          print(start_index, gnrCode)
          f_url = f"https://www.melon.com/genre/song_listPaging.htm?startIndex={start_index}&pageSize=50&gnrCode={gnrCode}&dtlGnrCode=&orderBy=NEW&steadyYn=Y"
          url = f_url
          res = requests.get(url, headers = headers)

          # 성공일 경우
          if res.status_code == 200:
              text = res.text

              bs = BeautifulSoup(res.text, 'html.parser')

              # rank 순위


              tr_lst = bs.select("tbody > tr")
              result = make_info_list(tr_lst, gnrCode)
              lst += result

              time.sleep(0.5)

          else:
            print('실패 res.status_code:', res.status_code)
            break


          time.sleep(0.5)


    return lst

In [ ]:
gnrCodeList = ['GN0100', 'GN0200', 'GN0300', 'GN0400', 'GN0500', 'GN0600', 'GN0700', 'GN0800']
lst = make_album_info_dict(gnrCodeList)

1 GN0100
51 GN0100
101 GN0100
151 GN0100
201 GN0100
251 GN0100
301 GN0100
351 GN0100
401 GN0100
451 GN0100
1 GN0200
51 GN0200
101 GN0200
151 GN0200
201 GN0200
251 GN0200
301 GN0200
351 GN0200
401 GN0200
451 GN0200
1 GN0300
51 GN0300
101 GN0300
151 GN0300
201 GN0300
251 GN0300
301 GN0300
351 GN0300
401 GN0300
451 GN0300
1 GN0400
51 GN0400
101 GN0400
151 GN0400
201 GN0400
251 GN0400
301 GN0400
351 GN0400
401 GN0400
451 GN0400
1 GN0500
51 GN0500
101 GN0500
151 GN0500
201 GN0500
251 GN0500
301 GN0500
351 GN0500
401 GN0500
451 GN0500
1 GN0600
51 GN0600
101 GN0600
151 GN0600
201 GN0600
251 GN0600
301 GN0600
351 GN0600
401 GN0600
451 GN0600
1 GN0700
51 GN0700
101 GN0700
151 GN0700
201 GN0700
251 GN0700
301 GN0700
351 GN0700
401 GN0700
451 GN0700
1 GN0800
51 GN0800
101 GN0800
151 GN0800
201 GN0800
251 GN0800
301 GN0800
351 GN0800
401 GN0800
451 GN0800


In [ ]:
# (50 * 8) * 10 = 4000
len(lst)

2886

In [ ]:
columns = ['gnrCode', 'rank', 'song_name', 'artist', 'album_id', 'song_id']

result_df = pd.DataFrame(lst, columns=columns)

In [ ]:
result_df

,gnrCode,rank,song_name,artist,album_id,song_id
0,GN0100,1,내 손을 잡아,아이유,1286252,3414749
1,GN0100,2,Timeless,SG 워너비,399659,1944399
2,GN0100,3,봄 사랑 벚꽃 말고,HIGH4 (하이포),2248659,4579468
3,GN0100,4,희재,성시경,28985,418168
4,GN0100,5,라라라,SG 워너비,376502,1854856
...,...,...,...,...,...,...
2881,GN0800,271,데미안,심규선 (Lucia),2257294,4653838
2882,GN0800,272,어느 날,10CM,2150728,3909667
2883,GN0800,273,3X4,가을방학,1035872,2960596
2884,GN0800,274,"우정, 그 씁쓸함에 대하여",10CM,1166956,3136469


In [ ]:
result_df[['gnrCode', 'song_id']].groupby('gnrCode').count()

,song_id
gnrCode,
GN0100,428
GN0200,481
GN0300,346
GN0400,338
GN0500,226
GN0600,439
GN0700,353
GN0800,275


In [ ]:
# csv 파일 생성
result_df.to_csv('장르별 스테디셀러.csv', encoding='utf-8', index=False)

In [ ]:
# csv 파일 읽기
pd.read_csv('장르별 스테디셀러.csv', encoding='utf-8')

,gnrCode,rank,song_name,artist,album_id,song_id
0,GN0100,1,내 손을 잡아,아이유,1286252,3414749
1,GN0100,2,Timeless,SG 워너비,399659,1944399
2,GN0100,3,봄 사랑 벚꽃 말고,HIGH4 (하이포),2248659,4579468
3,GN0100,4,희재,성시경,28985,418168
4,GN0100,5,라라라,SG 워너비,376502,1854856
...,...,...,...,...,...,...
2881,GN0800,271,데미안,심규선 (Lucia),2257294,4653838
2882,GN0800,272,어느 날,10CM,2150728,3909667
2883,GN0800,273,3X4,가을방학,1035872,2960596
2884,GN0800,274,"우정, 그 씁쓸함에 대하여",10CM,1166956,3136469
